# ES2 API Flow Example

This example shows end-to-end process workflow of ES2 Self-Hosted Python SDK.

## Import ES2

At very first time, you should install and import `es2` package to use python APIs. Before installing, make sure you have Python3.12 and virtual environment on your system. For more details, see `SDK installation` section in `Get Started`.

In [ ]:
# !pip install es2

In [ ]:
import es2
from es2.crypto import Cipher, KeyGenerator

## Key Control

### 1. Generate FHE Keys

To use APIs, we should generate Key, which is base of ciphertext of fully homomorphic encryption (FHE).

In [ ]:
key_path = "./keys"
# Key ID will be used to register key to ES2 server
# Key ID can be any string of length less than 20 characters
key_id = "test_key_id"
key_dir = f"{key_path}/{key_id}"

To generate keys, we need `KeyGenerator` instance. For the detailed usage, see `KeyGenerator` section in SDK API Reference. Now, we can generate keys! All keys will be saved to `key_path`. Secret Key will be generated and saved to `FRSecKey.bin` in `key_path`. Encryption and Evaluation key will be generated and saved to `EncKey.bin` and `EvalKey.bin`.

In [ ]:
# Generate keys
keygen = KeyGenerator(key_dir)

try:
    keygen.generate_keys()
except ValueError as e:
    print(f"Key generation failed: {e}")

We have generated key now, so let's generate `Cipher` with this key!


The `Cipher` class in ES2 provides a unified interface for both encryption and decryption of vectors and scores. You can provide the encryption and decryption key paths when initializing the `Cipher`, or you can specify them directly when calling the `encrypt` or `decrypt` methods. This flexibility allows you to manage keys according to your workflow and security requirements.


For example, you can initialize a `Cipher` with both key paths for convenience, or you can create a `Cipher` with only the dimension and provide the key paths at each operation. This makes it easy to use the same `Cipher` instance in different contexts or with different keys as needed.

In [ ]:
dim = 512

cipher = Cipher(dim=dim, enc_key_path=f"{key_dir}/EncKey.bin")

### 2. Connect to ES2 end-point server

First, we need to connect to ES2 end-point server.

In [ ]:
# Connect to the ES2 end-point server(default [ip:port] is [0.0.0.0:50050])
es2.init(address="0.0.0.0:50050", key_path=key_path, auto_key_setup=False)
# Check if the connection was successful
if es2.is_connected():
    print("Connected to ES2.")
    
else:
    print("Failed to connect to ES2.")

Now, we should register key to ES2 server.
Server cannot encrypt or decrypt, so we register evaluation key only.

In [ ]:
# Register eval key to ES2-MSA
es2.register_key(key_id)

If we wanna know specific information about the registered key, we can use `get_key_info` method.

In [ ]:
es2.get_key_info(key_id)

## 3. Load Data into ES2

### Create an Index

For encrypted similarity search, we first prepare a vector index, called `Index`, to store encrypted vectors and their metadata in the ES2 system. For the detailed usage, see the `Index` section in the Python SDK API Reference.

In [ ]:
index_name = "test_index"
index = es2.create_index(index_name, dim)

In [ ]:
# Check if the index creation was successful
index_lst = es2.get_index_list()
print(index_lst)

### 4. Run example with sample data

And also, we need test data.
For now, let's use some random data.
Now let's test with randomly generated sample data!
First, we need to create a new index to save encryptred vectors.

In [ ]:
import numpy as np

# Define a function to generate random vectors
def generate_random_vectors(dim):
    if dim <= 16 or dim > 4096:
        raise ValueError(f"Invalid dimension: {dim}.")
    
    vec = np.random.uniform(-1.0, 1.0, dim)
    norm = np.linalg.norm(vec)

    if norm > 0:
        vec = vec / norm

    return vec

num_data = 10

db_vectors = [
    generate_random_vectors(dim) for _ in range(num_data)
]

db_metadata = [f"Item {i+1}" for i in range(num_data)]

### Encrypt Vectors

To securely store or search vectors in the ES2 system, we need to encrypt them using generated encryption keys. See the `Cipher` in the Python SDK API Reference.

In [ ]:
# Encrypt vectors to be indexed
db_ctxt = []
db_metadata = []
for i, vec in enumerate(db_vectors):
    db_ctxt.append(cipher.encrypt(vec, "item"))
    db_metadata.append(f"Item {i+1}")

### Insert vectors into the index

Let's insert the encrypted vectors into the created index into the ES2 server. It sends a list of encrypted vectors and their metadata to the ES2 server and store them into a specified index.

In [ ]:
index.insert(db_ctxt, metadata=db_metadata)

## Encrypted Similarity Search

### Prepare query

First, prepare query for encrypted search.

In [ ]:
query_index = np.random.randint(0, num_data)
# query argument is a list of query vectors
query_vector = db_vectors[query_index]
print("Answer query index:", query_index + 1)

### Encrypted search on the index

Let's perform encrypted similarity search. Once all the encrypted vector index and query vectors are ready, we can now perform a similarity search on encrypted data without decrypting the data. For more details, see the `Index.search` in the API Reference.

In [ ]:
search_index = es2.Index(index_name)
result_ctxt = search_index.scoring(query_vector)[0]

### Decrypt search results

The result of the encrypted search (`result_ctxt`) is the ciphertext, which means that the server could have not seen the original values. To see the result, it requires to decrypt the results with the user's secret key. The encrypted search results include score vectors of each text embedding vectors, as the results of the encrypted similarity search, so that it provides the score vectors after decryption. For more details, see the `Cipher` in the Python SDK API Reference.

In [ ]:
result = search_index.decrypt_score(result_ctxt, sec_key_path=key_dir + "/SecKey.bin")
result

### Extract Top-k Relevance

To extract relevant vectors, we first identify the top-k indices from the decrypted result vectors. These indices correspond to the most similar vectors in the encrypted database.

In [ ]:
output_metadata = search_index.get_topk_metadata_results(result, top_k=1, output_fields=["metadata"])
output_metadata

In [ ]:
es2.drop_index(index_name)

In [ ]:
es2.release_key(key_id)